# Cython
Simon Funke

Date: **Sep 28, 2016**

## Contents

* Cython introduction
* An example
* Cython and NumPy
* Summary

##  Cython is a **superset** of Python

* Cython is a **superset** of Python, with additional functionality   for defining C types and calling C functions
* Cython generates C wrapper code, which is compiled into a Python   extension module
* Major advantage: enables incremental code optimization

## `cdef`  is used to declare C variables

```cython
cdef int i, j, k
cdef float f, g[42], *h
```

## Cython function definitions

There are three kinds of Cython function definitions: `def`, `cdef` and `cpdef`:

```cython
# Python function.
def foo(int i, char *s):
    
# C function. Not visible to Python code that imports the module 
cdef int eggs(int i, float f):  

# "Hybrid". Generates both Python and C functions.
cpdef double foo_2(int i, float f):

```

**Note**: Function arguments and return types may be declared. 

## Cython optimises based on type definitions  

* If no type is specified for a variable, parameter or return type, it defaults to a Python object
* The standard Python for-loop is used in Cython:

```cython
for i in range(n):
   ...
```   

* If `i` is declared as an integer (with `cdef int i`), this will be optimized into a standard C loop.

## A Cython example

* Approximate the integral of a general function `f(x)`
   <center>![xkcd](figs/num_itg.png "Integral")Integral of $f(x) = sin(x^2)$</center>

* Numerical integration: accuracy increases with number of intervals

* Speed is not a problem in 1D, but may be critical in 3D

## Cython example: Standard Python

Python implementation (not optimized) of the integration:

In [4]:
from math import sin

def f(x):
    return sin(x**2)

def integrate_f(a, b, N):
    s = 0
    dx = (b-a)/N
    for i in xrange(N):
        s += f(a+i*dx)
    return s * dx

Integration takes around 3.5 seconds with `N=1000000`.

## Cython example: Compilation

Our first Cython file `integral.pyx` is identical to the Python file
(Python code is legal Cython code).

Compile with

```bash
cython integral.pyx
gcc -fPIC $(pkg-config --cflags --libs python3) integral.c 
gcc -shared -o integral0.so integral.o
```

## Cython example: Compilation with distutils

Compiling with `distutils` is easier. 

Make a script `setup.py`:

```python
from distutils.core import setup, Extension
from Cython.Distutils import build_ext

setup(
    cmdclass={'build_ext': build_ext},
    ext_modules=[Extension("integral", ["integral.pyx"])]
)
```

and compile the module with

```bash
python setup.py build_ext --inplace
```

## Cython example: adding ctypes

* Simply compiling the Cython file gives only minor speedup: loop runs in C, but makes numerous calls to the Python/C API
* To have any real speedup, we need to introduce types:

```cython
from libc.math cimport sin

def f(x):            
    return sin(x**2)   

cpdef double integrate_f(double a, double  b, int N):
    cdef double s = 0
    cdef double dx = (b-a)/N
    cdef int i
    for i in range(N):  # compiles to C loop if i is declared as int
        s += f(a+i*dx)
    return s*dx
```

## Cython example: final version

* A fully typed version runs about 10 times faster:

```cython
cdef extern from "math.h":
     double sin(double arg)

cdef double f(double x):
    return sin(x**2)

cpdef double integrate_f(double a, double b, int N):
    cdef double s=0
    cdef double dx = (b-a)/N
    cdef int i
    for i in range(N):
        s += f(a+i*dx)
    return s * dx
```    

* Speedup can be much higher, but requires slightly more   complex example (loops within loops...)

## Cython example: Adding "more C" gives more speedup:

<table border="1">
<thead>
<tr><th align="center">       Implementation        </th> <th align="center">Timing (normalised) </th> </tr>
</thead>
<tbody>
<tr> <td align="center">       Pure Python        </td> <td align="center">1.0 </td> </tr>
<tr> <td align="center">   Cython, no types              </td> <td align="center">   0.74    </td> </tr>
<tr> <td align="center">   *double*                 </td> <td align="center">   0.64    </td> </tr>
<tr> <td align="center">   *double* + *int*    </td> <td align="center">   0.40    </td> </tr>
<tr> <td align="center">   Types and *math.h*       </td> <td align="center">   0.12    </td> </tr>
</tbody>
</table>

# Cython and numpy

Cython works with numpy arrays as well.

# Example: A pure Python version

Apply `sin` to all numbers in an array:

In [2]:
import numpy
from math import sin


def apply_sin(a):
    out = numpy.ndarray(len(a), dtype=numpy.double)

    for i in range(len(a)):
        out[i] = sin(a[i])

    return out

Usage:

In [3]:
a = numpy.linspace(0, 10, 1e6, dtype=numpy.double)
apply_sin(a)

array([  0.00000000e+00,   1.00000100e-05,   2.00000200e-05, ...,
        -5.44004329e-01,  -5.44012720e-01,  -5.44021111e-01])

# Declaring numpy data types

```cython
cdef numpy.ndarray[numpy.double_t, ndim=1] out
out = numpy.zeros(1000, dtype=numpy.double)
```

Note that the definition used the `cython` version of the data type. 

Translation table:

| Numpy datatype| Cython datatype|
| ------------- |:-------------:|
| numpy.int8      | numpy.int8_t |
| numpy.int16      | numpy.int16_t |
| numpy.single      | numpy.single_t |
| numpy.double      | numpy.double_t |
| numpy.complex      | numpy.complex_t |


# Declaring numpy data types

```cython
import numpy
cimport numpy

cdef extern from "math.h":
     double sin(double arg)

cpdef numpy.ndarray[numpy.double_t, ndim=1] apply_sin(numpy.ndarray[numpy.double_t, ndim=1] a):
    cdef int i

    cdef numpy.ndarray[numpy.double_t, ndim=1] out
    out = numpy.ndarray(len(a), dtype=numpy.double)

    for i in range(len(a)):
        out[i] = sin(a[i])

    return out
```

## Using the Cython-numpy module

Save this file as `apply.pyx`. Once compiled, the cython module can be used as:

```python
from apply import apply_sin

a = numpy.linspace(0, 10, 1e6, dtype=numpy.double)
out = apply.apply_sin(a)
```

## Timings

<table border="1">
<thead>
<tr><th align="center">       Implementation        </th> <th align="center">Timing (normalised) </th> </tr>
</thead>
<tbody>
<tr> <td align="center">       Pure Python        </td> <td align="center">1.0 </td> </tr>
<tr> <td align="center">   Cython                 </td> <td align="center">   0.205    </td> </tr>
<tr> <td align="center">   Numpy               </td> <td align="center">   0.202    </td> </tr>
</tbody>
</table>

## Cython summary

* Cython pros and cons
    * [+] Allows incremental optimization, easy to access C libraries, generated C code more compact and readable than swig, active developer community, advanced and flexible
    * [-] Not as quick as Instant for speeding up a single loop, less       suitable than Swig for wrapping large libraries to Python modules, fully optimized code not as readable as Python
* Should be considered (maybe as a first choice?) for mixing Python with C

## Remember

* Core examples can be found in <https://github.com/UiO-INF3331/code-snippets-16.git> in `mixed` folder:
    * Instant with numpy and 2D numpy
    * Swig with and without numpy
    * Cython with and without numpy

## What is the most convenient approach in each case?
* Instant for inlining a simple loop
* Cython for advanced Python/C interactions
* Swig for wrapping larger libraries